# 東森新聞雲爬蟲練習
## 練習從東森新聞雲網站中，爬取新聞摘要及詳細內容。
## 學習利用Selenium模擬人為操作，更新動態網頁後爬取新聞內容。

In [2]:
import requests
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

### 以財經新聞為例，先連結到財經新聞網頁，複製其URL。

In [4]:
ETtoday_url = "https://www.ettoday.net/news/focus/%E8%B2%A1%E7%B6%93/"  #財經新聞

### 用Selenium打開一個瀏覽器，連結到網站。然後模擬下拉網頁的動作，讓網頁完成更新，再處理後續動作。

In [5]:
browser = webdriver.Chrome()
browser.get(ETtoday_url)  # 打開瀏覽器並連到東森新聞雲網頁

SCROLL_PAUSE_TIME = 1

#
# 以下是用Selenium模擬下拉網頁動作，讓網頁更新
#
last_height = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
while True:
    #Scroll down to bottom
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    
    #Wait to load the page
    time.sleep(2)
    
    #Calculate new scroll height and compare with last scroll height
    new_height = browser.execute_script("window.scrollTo(0, document.body.scrollHeight)")
    
    if new_height == last_height:
        break
    last_height = new_height
    

### 到這裡網頁已經更新完畢，所有的新聞都已經出現在網頁上。接下來做爬取收集新聞的動作。

In [6]:
#
# 這裡先建立一個函數，其功能是連到外部連結，並爬取新聞內容。
#
def getNewsDetailContent(link_url):
    resp = requests.get(link_url)
    resp.encoding = 'utf-8'
    #print(resp.text)

    soup = BeautifulSoup(resp.text, 'lxml')
    news_content = soup.find("div", attrs={'class':'story'}).find_all("p")
    for p in news_content:
        """
        .string屬性說明：
        (1) 若當前tag節點底下沒有其他tag子節點，會直接抓取內容(返回"NavigableString")
        (2) 若當前tag節點底下只有唯一的一個tag子節點，也會直接抓取tag子節點的內容(返回"NavigableString")
        (3) 但若當前tag節點底下還有很多個tag子節點，.string就無法判斷，(返回"None")
        """
        if ((p.string) is not None):
            print(p.string)

### 解析HTML並萃取新聞摘要，若有外部的連結，再連到外部連結並把詳細新聞內容抓取下來。

In [7]:
# 爬取網頁內容，解析後萃取新聞摘要
html = browser.page_source
soup = BeautifulSoup(html, "lxml")
all_news = soup.find("div", attrs={'class':'block block_1 infinite_scroll'})

news_block = all_news.find_all('div', attrs={'class':'piece clearfix'})

for i, news_item in enumerate(news_block):
    print("----------------------------------------------------------------------")
    news_body = news_item.find('h3')
    print("\n[%d] %s\n" % (i, news_body.a.string))
    
    # 
    # 連到外部連結，擷取詳細新聞內容
    #
    externalLink = "https://www.ettoday.net" + news_body.a["href"]
    getNewsDetailContent(externalLink)


----------------------------------------------------------------------

[0] 新冠疫情衝擊三星營運　記憶體與面板台廠可望受惠轉單效益

▲外電報導三星旗下龜尾市工廠今（1）日確診一名病例，公司決定將暫時關閉工廠防疫。（圖／路透）
記者林淑慧／台北報導
新冠肺炎（COVID-19）疫情衝擊全球，南韓已經淪陷為重災區。依據外電報導，韓國重量級企業三星旗下龜尾市工廠今（1）日確診一名病例，公司決定將暫時關閉工廠，恐衝擊後續營運，法人表示，南韓電子業斷鏈危機升高，台灣記憶體與面板廠可望受惠轉單效益。
依據《路透》引述《韓聯社》報導指出，三星龜尾市工廠的一名員工，先前因確診新冠肺炎，已暫時關閉營運至24日，沒想到今天再度確診一名病例；而越南為了防範新型冠狀病毒，也宣佈隔離所有從南韓入境者。
隨著韓國疫情急遽升溫，法人表示，韓國為全球記憶體與面板主要供應國家，電子業斷鏈危機升高，各界轉為關注記憶體與面板關鍵零組件供應是否會出現短缺。
法人指出，晶圓廠自動化程度高，生產營運受疫情影響應有限，不過，若韓國疫情加劇，一旦記憶體與面板供應受阻，勢必將重創全球包括手機、個人電腦與電視等電子產品市場。
由於韓國電子業可能出現斷料風險，法人表示，包括電子產品製造廠及代理、通路商都可能緊急儲備庫存，也可能轉移生產基地。台灣記憶體與面板廠營運不排除可望受惠轉單效益。
----------------------------------------------------------------------

[1] 疫情衝擊商用房地產市場 　劉學龍：逢低進場，有另一波漲勢

圖文／CTWANT
新冠肺炎來勢洶洶，各行各業皆受到影響，間接影響商用房地產租售市場，高力國際董事總經理劉學龍認為，疫情將導致兩岸投資人觀望期拉長，短期內交易量將停滯，但待疫情稍緩或結束時，將有另一波漲勢，現階段房地產可能出現價格修正，有「逢低進場、危機入市」的機會點。
辦公大樓方面，劉學龍分析，企業租戶也可能放緩擴編或增租計畫，所幸台北市辦公大樓空置率目前仍在歷史低點，對於空置率及租金影響應不至太大。工業地產方面，由於受到中國大陸此波疫情影響，造成製造業供應斷鏈危機，廠商分散產能的策略將會持續轉移至東南亞或回流台灣，自用型買家可望繼續撐起市場動能。


▲口罩生產線上作業情形。（圖／記者湯興漢攝）
文／中央社記者吳柏緯台北29日電
行政院加碼新台幣9000萬元採購30條口罩產線，預計3月中下旬陸續交機。經濟部指出，在產能全開的情形下，4月上旬的口罩日產能可達1300萬片。
口罩實名制與新的生產線陸續投入生產後，一般民眾都可以順利在藥局購得口罩，目前台灣的口罩數量仍算充足，但為了未來可能發生社區感染情況，政府進行超前布署，除了原本採購的60條產線陸續交機、投入生產外，日前行政院再核定新台幣9000萬元增購30條生產線，希望多生產口罩，以防不時之需。
經濟部官員受訪指出，目前規劃新增的30條產線，已經開始遴選，將比照前一次60台產線的方式，分配到口罩工廠、安裝並投入生產。
官員進一步說明，截至昨天，第一波60台口罩產線已經有超過半數送交口罩工廠，建置並測試，將陸續投入生產，預計3月上旬完成所有交機、建設工作，並在3月中旬前全數投入生產。
全新的30台產線，官員表示，當遴選結果出爐後，預計將從3月中下旬開始交機，並在3月底前全數完成。經過安裝與測試，以及與業者既有的產線整合後，4月上旬預計可以陸續投入生產，屆時，在90條產線產能全開的情形下，台灣口罩的日產能可上看1300萬片。
至於原料是否足以供應，經濟部官員說明，根據日前進行原料盤點的結果，目前原料可以供應每日1500萬片的產能，並持續到6月底。未來若日產能增加至1300萬片，供應仍無虞。（編輯：張均懋）1090229
----------------------------------------------------------------------

[12] 新生兒保單規劃技巧曝光！專家幫整理「投保5大重點」：寶寶出生後7天內速保

▲寶寶剛出生，到底要購買什麼保單呢？（圖／pixabay）
記者楊絡懸／台北報導
新生兒到底該準備什麼保單，一直是新手爸媽最頭疼的問題，為此，錠嵂保險經紀人整理「新生兒5大保障重點」，透過簡單的觀念，了解新生兒投保須知與條件；而在新生兒投保前，也應該要先規劃保單內容與準備報戶口的資料。
為什麼寶寶剛出生，就要先買保險呢？錠嵂保經說明，新生兒投保條件嚴格，若有體況，保險公司就會延期或拒保，而且新生兒出生後6個月後，免疫力逐漸降低，一旦發生疾病或住院，保險公司就會認定有體況問題；因此，若一出生就投保，不但保費便宜，還可規劃較高的保

記者王致凱／綜合報導
新冠肺炎疫情引爆全球經濟恐慌，美股周五開盤持續跌勢，道瓊工業指數早盤暴跌超過800點，隨後跌勢持續擴大，標普500指數和那斯達克綜合指數也下跌超過3％。
美股本周創下金融危機以來最慘烈一周，道瓊斯工業指數下跌超過1000點或4％，跌破25,000點大關；標普500指數下跌4％，那斯達克綜合指數則下跌3％
美股相對於近期高點已經下跌10％以上；道指本周已經下跌近4000點，且跌勢仍在持續。
----------------------------------------------------------------------

[24] 沉寂27年彰化地標喬友大樓　無人櫃台百香果商旅掛牌

▲喬友大廈無人櫃台百香果商旅掛牌。（圖／彰化縣政府提供）
記者唐詠絮／彰化報導
沈寂27年的喬友大樓，高15樓是彰化市地標，12年前發生祝融致商家撤離，大廈管委會去年開始招商並在內部裝潢施工，8、9樓規劃做無人櫃台旅館，27日已經由縣府城市暨觀光發展處批核，未來此家「百香果商旅旅館」，將採用最先進的物聯網系統控管，預定3月下旬開始試營運，勢必將帶動彰化火車站前的商機。
矗立在彰化市火車站前的喬友大廈，在1993年開幕時，這棟地下3樓、地上15樓的建物，包括有百貨商場、KTV、遊樂世界、冰宮、旅館等，繁榮至極，一度是除八卦山大佛之外，彰化市最熱鬧的地標。
去年大廈所有權人組管委會展開重修活化自救，傳出B1-B3規劃地下停車場外，還有東南亞異國等料理美食，而其他樓層則出租世界健身中心、補習班及AI智能無人櫃台旅館等，重新營運後喬友大廈，一定可以帶給市民一個充滿健康朝氣有活力的新氣象。
縣府城市暨觀光發展處表示，喬友大樓未來8、9樓規劃做無人櫃台旅館，採用最先進的物聯網系統控管，預估房間數，單人房（可住1-2人）有24間，雙人房（可住3-4人）有23間，總房間數47間，將服務更多南來北往的鄉親朋友，未來縣府將續予輔導，引進更多優質行業進駐，藉由喬友大樓的重新整裝將帶動火車站附近的商機，有人潮就有錢潮，相信活化後的大樓會讓人耳目一新。
----------------------------------------------------------------------

[25] 防疫優先！經濟部：專利線上申請價「愛的迫降」減免600元起

▲經濟部智慧局表

----------------------------------------------------------------------

[36] 太陽能產業景氣低迷　寶德電化材料聲請破產

▲太陽能板。（圖／記者湯興漢攝）
記者姚惠茹／台北報導
由國發基金及華新麗華等民股合資的寶德能源（寶德電化材料）26日宣布，受到太陽能市場競爭白熱化，再加上新冠肺炎（COVID-19）疫情而導致資金未能到位，公司帳面資金已不足清償現有債務，董事會決議向法院聲請破產。
寶德電化材料公告指出，由於缺乏營運資金，增資遍尋投資人未果，公司現有存款不足，2月25日屆期的銀行利息1,923萬元、2月26日屆期的支票664萬元均無法支付，另有債權人於2月25日以債權金額154萬元，就部分資產進行動產查封的強制執行事宜，已無法繼續營運，因此聲請破產。
寶德公司於2010年10月成立，股東除了國發基金、中技社、耀華玻璃等，還有民股華新麗華，主要生產高品質太陽能級多晶矽，但是受到太陽能製造產業低迷，轉型生產半導體級多晶矽，但是半導體產業驗證程序冗長，導致公司供貨前缺乏資金維繫運作
寶德電化材料聲請破產，大股東國發基金等官股機構估計將損失30億元，華新麗華持有寶德持股22.46％，估計要認列16.8億元損失，並由於寶德電化材料還有120億元的聯貸，因此參與聯貸的銀行也將認列呆帳損失。
----------------------------------------------------------------------

[37] 國人平均有3.16張健康險保單…高達54%額度僅2千元！壽險公會「3大建議」檢視保障

▲用以支付「住院醫療費用」的健康險保單相當重要。（圖／pixabay）
記者楊絡懸／台北報導
台灣民眾持有健康險商品，從1995年平均0.4件，至2019年已增至平均3.16件，然而個人健康險的日額給付，竟有高達54.6%國人都在2000元以下，且投保長期照護險的比率僅3.03%。為此，壽險公會提出3大建議，民眾應及早檢視保障額度是否充足。
壽險公會引述保險事業發展中心統計資料，國人1993年投保健康險約409萬件，在1995年實施全民健保制度後成長至862萬件，直至2019年已達7466萬件；若以當年度全國人口數為分母，1993年國人平均持有健康險保單約0.19件，1995

▲經濟部表示，國內口罩的日均產量，預計三月能達到1000萬片以上。（圖／經濟部提供）
記者林淑慧／台北報導
行政院會今（27）日通過政府新增30條口罩產線，為提升口罩產能，經濟部長沈榮津幾乎天天到到產線的製造廠，現地追製造的進度，今天更與行政院副院長陳其邁請到最前線，宣示力拚日均產能上看1300萬片的決心。
新冠肺炎（COVID-19）疫情延燒，口罩成為防疫重點物資，為滿足防疫需求，經濟部斥資近2億元新建口罩產線，自15日起新機台陸續測試上線並投入生產，行政院今天再拍板添購30台口罩機台，預計下月9日起日均產能可達1千萬片，30台新機投入之後，口罩日均產能更上看1300萬片。
經濟部官員指出，要在短時間內完成一條條口罩新產線，並不是變魔術，是靠政府民間團結努力，才能每一天都進步，被同仁暱稱為「歐吉桑部長」的沈榮津以近40年的經歷督軍，「他不只要在辦公室指揮，更要直接到最前線。」
經濟部表示，過去只有個位數員工的口罩產線製造廠，現在有上百位工具機產業、法人的業界精英進駐協助，大家一起逐一確認口罩產線的料件，解決製造時的疑難雜症，確保能如期交機，儘快提升口罩產能。
----------------------------------------------------------------------

[47] 新冠肺炎疫情升級　13家銀行啟動「異地辦公」防疫

▲金管會。（圖／記者紀佳妘攝）
記者紀佳妘／台北報導
新冠肺炎（COVID-19）疫情蔓延，行政院宣布防疫升級，中央疫情指揮中心提升為一級開設。金管會今（27）日表示，初步了解，已有13家銀行以「異地辦公」的方式因應，目前尚無人確診，人員都是安全的。
隨著防疫升級，銀行是否會有新的因應措施，金管會銀行局官員表示，「營運不中斷」是銀行重要的一環，金管會在日常監理上，已要求銀行須有營運不中斷的計畫，因應天災人禍。
針對防疫升級，銀行局官員指出，銀行維持營運不中斷的計畫，都有納為考量，金管會也會持續關注銀行後續的防範與作為，目前尚無人確診，所有人員都是安全的。
根據金管會初步了解，已有13家銀行採取「異地辦公」方式因應，大多都以總行開始做起。至於金管會是否會採取異地辦公？官員則說，公務機關已有積極準備、超前部署，將會配合行政院指示辦理。
此外，為加強銀行防疫措施，金管會先前也祭出5措施，確保營運不中斷，首先是從1

In [8]:
browser.quit()